In [1]:
%load_ext autoreload
%autoreload 2
from src.datasets.mimic import make_mimic
from src.models.vision_transformer import PatchEmbed
import torch
if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"

In [2]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="initial-tests",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arnoseeliger (idp-sjepa). Use `wandb login --relogin` to force relogin


acc,▁▆▇▇▇▇██
loss,█▅▃▅▄▃▁▁
acc,0.83942
loss,0.10735


In [11]:
dataset, data_loader = make_mimic(None,1)

In [12]:
iterator = enumerate(data_loader)

In [13]:
idx, data = next(iterator)
data = data.to(device)
data.size()

torch.Size([1, 12, 5000])

In [14]:
embedder =  PatchEmbed(sig_size=5000, patch_size=100, in_chans=12, embed_dim=768).to(device)
embedder(data).shape

torch.Size([1, 600, 768])

In [15]:
from src.models.vision_transformer import VisionTransformer
vit = VisionTransformer(img_size=[5000], patch_size=100, in_chans=12, embed_dim=768).to(device)

600
torch.Size([1, 600, 768])
torch.Size([1, 600, 768])


In [17]:
out = vit.forward(data)
print(out.shape, out)

torch.Size([1, 600, 768]) tensor([[[-1.6104, -1.2473, -0.7055,  ...,  0.2991,  1.0297,  0.4338],
         [-0.2477,  0.0807,  0.2415,  ...,  0.0690,  0.9897,  0.5701],
         [ 0.1498,  0.5065,  0.5374,  ..., -0.0860,  1.0420,  0.6665],
         ...,
         [ 0.0716, -0.2187, -1.3726,  ..., -0.4497,  1.4174,  0.6215],
         [-1.2425,  0.0398, -0.8168,  ..., -0.4377,  1.6730,  0.4678],
         [-1.8128, -0.7367,  0.1478,  ..., -0.4691,  1.8506,  0.5096]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)


In [19]:
from src.models.vision_transformer import VisionTransformerPredictor
pred = VisionTransformerPredictor(num_patches=600, embed_dim=768).to(device)

In [20]:
out2 = pred.forward(out)
print(out2.shape, out2)

TypeError: VisionTransformerPredictor.forward() missing 2 required positional arguments: 'masks_x' and 'masks'